In [1]:
import os
os.chdir('..')

In [2]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
import orjson
from pathlib import Path

from src.models.prompt import Prompt
load_dotenv()

True

In [3]:
data = [
    orjson.loads(i)
    for i in Path('data/raw/example01.jsonl').read_text().split('\n')[:-1]
]
sample = data[32]
sample

{'label': 1,
 'news': '東協優選【王郁倫╱台北報導】新興市場股票基金受惠經濟成長快速，前景看好，泰國在先泰皇蒲美蓬的1年國殤期結束之後，觀光商機今年可望更勝去年，加上軍政府積極推出多項經濟刺激政策，推動民選總統吸引外資，今年泰國經濟成長可期，專家建議由於國際資金流動快速，可逢泰國股市回跌逢低分批布局。安聯中國東協基金暨安聯亞洲動態策略基金經理人許志偉表示，泰國在2017年11月剛剛結束1年國殤期，預估將重啟觀光熱潮，泰國是東協國家中賺觀光財比例最高的國家，觀光佔整體國家GDP比例高達18%，但國殤期間禁止推廣促銷旅遊，所以10月底國殤結束後，開始觀光客回流明顯。許志偉表示，泰國國殤期結束後，來客量逐月有10%成長率，帶來龐大觀光財，根據統計，2017年觀光客年增率8.3%，今年預估觀光來客人數年成長率上看9.5%，以中國、俄羅斯、東西歐等地區觀光客最多。不過，雖觀光發展熱絡，但元大新東協平衡基金經理人吳昱聰提醒，泰股漲跌快，幅度大，基本面復甦，但風險面則有美國升息使資金回流美國的風險，若資金從泰國股債同時撤出，或泰國通膨過大，央行也不排除在下半年升息，這部分讓泰國債券隱憂高於股票。許志偉則補充，泰國債券以泰銖發行，外商不青睞，所以仍建議股優於債。許志偉表示，泰國除觀光復甦，在經濟面也有許多利多條件，比方將興建中泰鐵路與泛亞鐵路，曼谷東南方機場有道路建設，距離曼谷東南方100公里處規劃東部經濟走廊，加強港口機場與道路建設，提供外商投資租稅優惠，由於泰國也是東協國家中汽車製造重鎮，預期對汽車零組件廠商有吸引力。泰國股市今年溫和上漲，主要產業包括金融、能源、稻米、橡膠、工業設備與汽車零組件及製造業，根據研調機構預測，今年上述產業將有雙位數百分點成長。對於2019年大選結果是否影響股市表現？許志偉則分析，對股市沒有太多加減分，但對經濟國貿發展有正面作用，主要是因為泰國過去60年來政權發展多次更迭，股市對政權轉換的敏感度已低，但民選總理出爐應該對國家信評提高，有正面加分。許志偉說，新興市場表現來說，東協比東歐政局穩定，相比之下，拉美政治不確定性高，東協市場成為投資相對優選，加上軍政府提出10年經濟計劃「泰國4.0」，除東部經濟走廊，也鼓勵外商投資，加速基礎建設及高科技招商，基本面相對穩健，根據統計，2017年泰股企業獲利年增7.5%，今年上看8.1%，2

In [9]:
prompt = Prompt()
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 0.2)
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)

chat_prompt_template = (
    ChatPromptTemplate
    .from_messages([
        ("system", prompt.system_message),
        MessagesPlaceholder(variable_name = "chat_history"),
        ("human", prompt.human_message),
    ])
    .partial(
        country = prompt.country,
        output_instructions = prompt.output_instructions_with_reason
    )
)
chain = LLMChain(
    prompt = chat_prompt_template.partial(news = sample["news"]),
    llm = llm,
    memory = memory,
    verbose = True
)

In [5]:
warning = {
    1: 'This news should be excluded, and thus the "pred" key should be 1. Organize the information and give a reronable interpretation',
    0: 'This news should not be excluded, and thus the "pred" key should be 0. Organize the information and give a reronable interpretation',
}

In [6]:
chain.run(correct_instructions=warning[sample["label"]])



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    I am an economist working on constructing Taiwan's Economic Policy Uncertainty Index (EPU index).
My primary goal is to classify wheter a news should be excluded when constructing EPU index in Taiwan.
There are two criteria I'm considering to exclude a news.

Criterion1:
The main idea of the news is either historical accounts or abstract subjective inferences, which won't impact Taiwan's economics for sure.
Hence, this kind of news should be excluded.

Criterion2:
There main idea of the news is not related with Taiwan.
For example, the people or companies mentioned in the news have nothing to do with Taiwan or the events
in the news don't actually happen within Taiwan. I will excluded the news as well.

Notice that you can first justify wheter there is a person, company or event in news related to Taiwan. 
If there isn't any, it should be excluded with high probability.

Human: 
Help me complete the classification

'{"pred": 0, "reason": "The news contains information about the economic growth and tourism prospects in Thailand, which could potentially impact Taiwan\'s economy and should not be excluded."}'

In [7]:
i = 66
sample = data[67]
sample

{'label': 1,
 'news': '外銷亮眼【張嘉伶╱台北報導】在中國從事髮飾、梳鏡與女性配飾外銷的弘帆（8433），昨日舉行上櫃前法說。董事長朱鵬飛表示，明年訂單明朗，還增不少新客戶，並決定漲價3~5%，因此，雖大環境不景氣，預計明年將是穩定成長的一年。弘帆預計12月19日上櫃，成百貨類股新兵。法人估，掛牌價約30元左右。朱鵬飛表示，營運是採接單生產，從事髮飾、梳鏡與女性配飾等代工設計，目前已開始設計2013年春季新品，並接明年聖誕節訂單，目前看來，明年訂單大致底定。但論景氣，朱鵬飛觀察，部分客戶保持觀望心態，一批訂單下完即會休息一下，可以預見明年全球景氣仍不是太熱絡。朱鵬飛說，前3大客戶包括Goody、Conair、Scunci，同時也是美國3大著名髮飾品廠商；早期公司100%業務皆來自於Goody，但為分散風險，13年前積極開發Conair客戶，目前也發展成第2大客戶。因前3大客戶佔營收比重6~7成，因此弘帆以開拓新的貿易客戶為首要任務，估計明年將有來自歐、美、日等地的新客戶訂單。另，弘帆也擴大發展自有通路，目前在中國擁有「綺麗生活館」12個據點，明年5月起將大舉展店，朱鵬飛指出，2016年計劃展店至100家，長期不排除讓綺麗生活館獨立掛牌。弘帆從製造貿易切入通路，綺麗生活館都設在百貨公司內，首店位於天津遠東百貨。朱鵬飛說，弘帆不開街邊店，可減人事與政策動盪問題，而且獲利回收的速度較快。目前綺麗生活館已有12個據點，鎖定華東、華北、東北、華中、華南及西南等6大地區，以及北京、上海等2大城市。雖綺麗生活館佔弘帆的營收比重僅2%，但朱鵬飛預計，明年隨著據點與營收成長，可望提升到10%。觀察近3年營運績效，弘帆年每年合併營收約20~22億元間，毛利率從2010年20.78%下滑至2011年的18.6%，今年前3季毛利率18.9%。朱鵬飛指出，主要是近年歷經金融海嘯、歐債等問題，公司未漲產品價格，因而壓縮毛利率，但公司已決定明年元月起漲價3~5%，預計將帶領合併毛利率回到19%以上。同時，弘帆在東筦坐擁12棟大樓，除部分樓層出租給外商企業，朱鵬飛也說，估未來1年可收租金上億元，對本業獲利將有不少幫助。弘帆今年前3季稅後純益6645萬元，每股純益(Earnings Per Share，EPS) 1.11元，低於去年同期的2.02元。主要是弘帆第3季時，一次性認

In [10]:
prompt.reasoning_instance(sample["news"], chat_prompt_template, llm, memory, sample["label"], warning, i)

{'pred': 1,
 'reason': "The news is about the export performance and business strategies of a company in China, which is not directly related to Taiwan's economy."}